In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

with open('vac.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))#set将文章中的所有不同字符取出，然后sorted排序
vocab_to_int = {c: i for i, c in enumerate(vocab)}#排好序的字符列表进行字典索引
int_to_vocab = dict(enumerate(vocab))#与上字典相反，索引号为键，字符为值
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)#把text中所有字符进行数字编码


def get_batches(arr, n_seqs, n_steps):

    # 用sequence和step计算batch大小，得出batch个数，最后不够一个batch的扔掉
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    arr = arr[:n_batches * characters_per_batch]

    # 重新reshape为sequence行，列数自动生成（-1）
    arr = arr.reshape((n_seqs, -1))

    # 生成样本特征batch及目标值batch（目标值为样本值的下一个字母）
    for n in range(0, arr.shape[1], n_steps):
        x = arr[:, n:n+n_steps]
        y = np.zeros_like(x)
        # 目标值往下滚动一个字母，目标batch最后一列可设置为样本特征batch的第一列，不会影响精度
        y[:, :-1], y[:,-1] = x[:, 1:], x[:, 0]

        # x,y为生成器（generater）
        yield x, y

def build_inputs(batch_size, num_steps):
    '''batch_size是每个batch中sequence的长度（batch行数）
        num_steps是batch列数
    '''
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return inputs, targets, keep_prob


In [2]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):

    # 创建LSTM单元

    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop


    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)

    return cell, initial_state

def build_output(lstm_output, in_size, out_size):

    # reshape
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])

    # 将RNN输入连接到softmax层
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))

    logits = tf.matmul(x, softmax_w) + softmax_b
    out = tf.nn.softmax(logits, name='predictions')

    return out, logits

def build_loss(logits, targets, lstm_size, num_classes):

    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())

    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)

    return loss

def build_optimizer(loss, learning_rate, grad_clip):

    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))

    return optimizer

class CharRNN:

    def __init__(self, num_classes, batch_size=64, num_steps=50, lstm_size=128,
                num_layers=2, learning_rate=0.001, grad_clip=5, sampling=False):

        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()

        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        x_one_hot = tf.one_hot(self.inputs, num_classes)

        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state

        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)

        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)
        
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers =2
learning_rate = 0.0001
keep_prob = 0.5

In [3]:
epochs = 20

# 每200步保存一个checkpoint
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps, 
               lstm_size=lstm_size, num_layers=num_layers, 
               learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)#  the maximum number of recent checkpoint files to keep.  As new files are created, older files are deleted.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    counter = 0
    for e in range(epochs):
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss,
                                                 model.final_state,
                                                 model.optimizer],
                                                 feed_dict=feed)
            end = time.time()
            print('Epoch:{}/{}...'.format(e+1, epochs),
                  'Training Step:{}...'.format(counter),
                  'Training loss:{:.4f}...'.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))

            if (counter % save_every_n == 0):
                saver.save(sess, 'checkpoints/i{}.ckpt'.format(counter))

    saver.save(sess, 'checkpoints/i{}.ckpt'.format(counter))

Epoch:1/20... Training Step:1... Training loss:4.5800... 1.1901 sec/batch
Epoch:1/20... Training Step:2... Training loss:4.5719... 0.2016 sec/batch
Epoch:1/20... Training Step:3... Training loss:4.5633... 0.2107 sec/batch
Epoch:1/20... Training Step:4... Training loss:4.5535... 0.1921 sec/batch
Epoch:1/20... Training Step:5... Training loss:4.5439... 0.2251 sec/batch
Epoch:1/20... Training Step:6... Training loss:4.5319... 0.1941 sec/batch
Epoch:1/20... Training Step:7... Training loss:4.5187... 0.2006 sec/batch
Epoch:1/20... Training Step:8... Training loss:4.5033... 0.2020 sec/batch
Epoch:1/20... Training Step:9... Training loss:4.4861... 0.2078 sec/batch
Epoch:1/20... Training Step:10... Training loss:4.4583... 0.2131 sec/batch
Epoch:1/20... Training Step:11... Training loss:4.4295... 0.2107 sec/batch
Epoch:1/20... Training Step:12... Training loss:4.3857... 0.1948 sec/batch
Epoch:1/20... Training Step:13... Training loss:4.3276... 0.1961 sec/batch
Epoch:1/20... Training Step:14... 

Epoch:1/20... Training Step:111... Training loss:3.3065... 0.2036 sec/batch
Epoch:1/20... Training Step:112... Training loss:3.2771... 0.1931 sec/batch
Epoch:1/20... Training Step:113... Training loss:3.3127... 0.1936 sec/batch
Epoch:1/20... Training Step:114... Training loss:3.2938... 0.1959 sec/batch
Epoch:1/20... Training Step:115... Training loss:3.3006... 0.1949 sec/batch
Epoch:1/20... Training Step:116... Training loss:3.3049... 0.2016 sec/batch
Epoch:1/20... Training Step:117... Training loss:3.3420... 0.1998 sec/batch
Epoch:1/20... Training Step:118... Training loss:3.3138... 0.1912 sec/batch
Epoch:1/20... Training Step:119... Training loss:3.3240... 0.2176 sec/batch
Epoch:1/20... Training Step:120... Training loss:3.3214... 0.2346 sec/batch
Epoch:1/20... Training Step:121... Training loss:3.3173... 0.1954 sec/batch
Epoch:1/20... Training Step:122... Training loss:3.3343... 0.2047 sec/batch
Epoch:1/20... Training Step:123... Training loss:3.3351... 0.1919 sec/batch
Epoch:1/20..

Epoch:1/20... Training Step:220... Training loss:3.2583... 0.1941 sec/batch
Epoch:1/20... Training Step:221... Training loss:3.2743... 0.2093 sec/batch
Epoch:1/20... Training Step:222... Training loss:3.2911... 0.1937 sec/batch
Epoch:1/20... Training Step:223... Training loss:3.2907... 0.1920 sec/batch
Epoch:1/20... Training Step:224... Training loss:3.2639... 0.2083 sec/batch
Epoch:1/20... Training Step:225... Training loss:3.2533... 0.1960 sec/batch
Epoch:1/20... Training Step:226... Training loss:3.2811... 0.2011 sec/batch
Epoch:1/20... Training Step:227... Training loss:3.2818... 0.1972 sec/batch
Epoch:1/20... Training Step:228... Training loss:3.2534... 0.1956 sec/batch
Epoch:1/20... Training Step:229... Training loss:3.2605... 0.2047 sec/batch
Epoch:1/20... Training Step:230... Training loss:3.2251... 0.1909 sec/batch
Epoch:1/20... Training Step:231... Training loss:3.2281... 0.1976 sec/batch
Epoch:1/20... Training Step:232... Training loss:3.2639... 0.2055 sec/batch
Epoch:1/20..

Epoch:2/20... Training Step:328... Training loss:3.2311... 0.1962 sec/batch
Epoch:2/20... Training Step:329... Training loss:3.2072... 0.1904 sec/batch
Epoch:2/20... Training Step:330... Training loss:3.1997... 0.2005 sec/batch
Epoch:2/20... Training Step:331... Training loss:3.2281... 0.2103 sec/batch
Epoch:2/20... Training Step:332... Training loss:3.2170... 0.2000 sec/batch
Epoch:2/20... Training Step:333... Training loss:3.2370... 0.2055 sec/batch
Epoch:2/20... Training Step:334... Training loss:3.2247... 0.2102 sec/batch
Epoch:2/20... Training Step:335... Training loss:3.2230... 0.1919 sec/batch
Epoch:2/20... Training Step:336... Training loss:3.2455... 0.1925 sec/batch
Epoch:2/20... Training Step:337... Training loss:3.2342... 0.2053 sec/batch
Epoch:2/20... Training Step:338... Training loss:3.2270... 0.1983 sec/batch
Epoch:2/20... Training Step:339... Training loss:3.2028... 0.2064 sec/batch
Epoch:2/20... Training Step:340... Training loss:3.2248... 0.1924 sec/batch
Epoch:2/20..

Epoch:2/20... Training Step:436... Training loss:3.0989... 0.2020 sec/batch
Epoch:2/20... Training Step:437... Training loss:3.0555... 0.1932 sec/batch
Epoch:2/20... Training Step:438... Training loss:3.0624... 0.1936 sec/batch
Epoch:2/20... Training Step:439... Training loss:3.0783... 0.2021 sec/batch
Epoch:2/20... Training Step:440... Training loss:3.1212... 0.2213 sec/batch
Epoch:2/20... Training Step:441... Training loss:3.0744... 0.1999 sec/batch
Epoch:2/20... Training Step:442... Training loss:3.0705... 0.1997 sec/batch
Epoch:2/20... Training Step:443... Training loss:3.0738... 0.1921 sec/batch
Epoch:2/20... Training Step:444... Training loss:3.0715... 0.1914 sec/batch
Epoch:2/20... Training Step:445... Training loss:3.0686... 0.1920 sec/batch
Epoch:2/20... Training Step:446... Training loss:3.0752... 0.2096 sec/batch
Epoch:2/20... Training Step:447... Training loss:3.0926... 0.2053 sec/batch
Epoch:2/20... Training Step:448... Training loss:3.0728... 0.1917 sec/batch
Epoch:2/20..

...

Epoch:19/20... Training Step:5346... Training loss:2.0152... 0.2042 sec/batch
Epoch:19/20... Training Step:5347... Training loss:2.0270... 0.1923 sec/batch
Epoch:19/20... Training Step:5348... Training loss:2.0287... 0.1989 sec/batch
Epoch:19/20... Training Step:5349... Training loss:2.0359... 0.2015 sec/batch
Epoch:19/20... Training Step:5350... Training loss:2.0683... 0.1939 sec/batch
Epoch:19/20... Training Step:5351... Training loss:2.0233... 0.1933 sec/batch
Epoch:19/20... Training Step:5352... Training loss:2.0159... 0.1924 sec/batch
Epoch:19/20... Training Step:5353... Training loss:2.0389... 0.2157 sec/batch
Epoch:19/20... Training Step:5354... Training loss:2.0443... 0.1929 sec/batch
Epoch:19/20... Training Step:5355... Training loss:2.0189... 0.1950 sec/batch
Epoch:19/20... Training Step:5356... Training loss:1.9842... 0.2030 sec/batch
Epoch:19/20... Training Step:5357... Training loss:1.9953... 0.2067 sec/batch
Epoch:19/20... Training Step:5358... Training loss:1.9920... 0.1

Epoch:19/20... Training Step:5453... Training loss:1.9470... 0.2009 sec/batch
Epoch:19/20... Training Step:5454... Training loss:1.9527... 0.1917 sec/batch
Epoch:19/20... Training Step:5455... Training loss:1.9694... 0.2110 sec/batch
Epoch:19/20... Training Step:5456... Training loss:1.9913... 0.2014 sec/batch
Epoch:19/20... Training Step:5457... Training loss:1.9462... 0.1947 sec/batch
Epoch:19/20... Training Step:5458... Training loss:1.9885... 0.2001 sec/batch
Epoch:19/20... Training Step:5459... Training loss:1.9369... 0.1927 sec/batch
Epoch:19/20... Training Step:5460... Training loss:1.9593... 0.2134 sec/batch
Epoch:19/20... Training Step:5461... Training loss:1.9626... 0.2001 sec/batch
Epoch:19/20... Training Step:5462... Training loss:2.0039... 0.1992 sec/batch
Epoch:19/20... Training Step:5463... Training loss:2.0192... 0.1936 sec/batch
Epoch:19/20... Training Step:5464... Training loss:1.9867... 0.2090 sec/batch
Epoch:19/20... Training Step:5465... Training loss:2.0002... 0.1

Epoch:20/20... Training Step:5560... Training loss:2.0081... 0.1957 sec/batch
Epoch:20/20... Training Step:5561... Training loss:1.9639... 0.1939 sec/batch
Epoch:20/20... Training Step:5562... Training loss:1.9723... 0.2069 sec/batch
Epoch:20/20... Training Step:5563... Training loss:1.9814... 0.1929 sec/batch
Epoch:20/20... Training Step:5564... Training loss:1.9288... 0.1922 sec/batch
Epoch:20/20... Training Step:5565... Training loss:1.9499... 0.1938 sec/batch
Epoch:20/20... Training Step:5566... Training loss:1.9514... 0.2152 sec/batch
Epoch:20/20... Training Step:5567... Training loss:1.9559... 0.2058 sec/batch
Epoch:20/20... Training Step:5568... Training loss:1.9641... 0.1937 sec/batch
Epoch:20/20... Training Step:5569... Training loss:1.9840... 0.2154 sec/batch
Epoch:20/20... Training Step:5570... Training loss:1.9794... 0.2064 sec/batch
Epoch:20/20... Training Step:5571... Training loss:1.9845... 0.1932 sec/batch
Epoch:20/20... Training Step:5572... Training loss:1.9580... 0.1

Epoch:20/20... Training Step:5667... Training loss:1.9660... 0.1972 sec/batch
Epoch:20/20... Training Step:5668... Training loss:1.9615... 0.1922 sec/batch
Epoch:20/20... Training Step:5669... Training loss:1.9598... 0.1948 sec/batch
Epoch:20/20... Training Step:5670... Training loss:1.9475... 0.2029 sec/batch
Epoch:20/20... Training Step:5671... Training loss:1.9325... 0.1936 sec/batch
Epoch:20/20... Training Step:5672... Training loss:1.9439... 0.1944 sec/batch
Epoch:20/20... Training Step:5673... Training loss:1.9378... 0.1936 sec/batch
Epoch:20/20... Training Step:5674... Training loss:1.9929... 0.2037 sec/batch
Epoch:20/20... Training Step:5675... Training loss:1.9859... 0.2065 sec/batch
Epoch:20/20... Training Step:5676... Training loss:2.0068... 0.2018 sec/batch
Epoch:20/20... Training Step:5677... Training loss:2.0083... 0.2008 sec/batch
Epoch:20/20... Training Step:5678... Training loss:2.0186... 0.1940 sec/batch
Epoch:20/20... Training Step:5679... Training loss:1.9616... 0.1

Epoch:20/20... Training Step:5774... Training loss:1.8888... 0.1936 sec/batch
Epoch:20/20... Training Step:5775... Training loss:1.9232... 0.1977 sec/batch
Epoch:20/20... Training Step:5776... Training loss:1.9297... 0.1982 sec/batch
Epoch:20/20... Training Step:5777... Training loss:1.9770... 0.1910 sec/batch
Epoch:20/20... Training Step:5778... Training loss:1.9346... 0.2001 sec/batch
Epoch:20/20... Training Step:5779... Training loss:1.9203... 0.1930 sec/batch
Epoch:20/20... Training Step:5780... Training loss:1.9296... 0.1991 sec/batch
Epoch:20/20... Training Step:5781... Training loss:1.9257... 0.2011 sec/batch
Epoch:20/20... Training Step:5782... Training loss:1.9690... 0.2150 sec/batch
Epoch:20/20... Training Step:5783... Training loss:1.9972... 0.2013 sec/batch
Epoch:20/20... Training Step:5784... Training loss:1.9122... 0.1968 sec/batch
Epoch:20/20... Training Step:5785... Training loss:1.9717... 0.2003 sec/batch
Epoch:20/20... Training Step:5786... Training loss:1.9518... 0.2

In [4]:
tf.train.get_checkpoint_state('checkpoints')
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]]=0
    p = p / np.sum(p)
    char = np.random.choice(vocab_size, 1, p=p)[0]
    return char

def sample(checkpoint, n_samples, lstm_size, vocab_size, prime='The '):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0, 0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction,
                                          model.final_state],
                                          feed_dict=feed)
            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])

            for i in range(n_samples):
                x[0,0] = c
                feed = {model.inputs: x,
                        model.keep_prob: 1.,
                        model.initial_state: new_state}
                preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

                c = pick_top_n(preds, len(vocab))
                samples.append(int_to_vocab[c])

    return ''.join(samples)

checkpoint = tf.train.latest_checkpoint('checkpoints')
print(checkpoint)
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime='The')
print(samp)

checkpoints/i5800.ckpt
INFO:tensorflow:Restoring parameters from checkpoints/i5800.ckpt
TheIN'TEN TATESLINS 

"I what do you diveny hade to get to book.

"There to you're go all the plays, she sale a prissting of meromant, to cent to hiss to to a tower any wolkn the plice. The pirllarse was a bust tame. 
The moner the cealling was and said, "How man, I'd going to take to but it it would and was the cars. The said, the mun who deart time a seed were have and he was stile at hear his and says "Why as the better boy whone he says a letale said, "What is she's the buct with shat into to be a chirdes thought of a strock to the somenter, and hus been the berided her hel and heard a bring said."

"This is a steation of the bort and went. The burn when houst a truck on she wenter. The stomp and then the women and theme aspiced, "Ind a betione a let on the steep it with the some are stonting and soush a can what the strecent of the sentoon on the partoot all and his for her then tous and what h

In [ ]:
mv /root/code/nmt/Untitled  